In [1]:
from dotenv import load_dotenv,dotenv_values
import json
import os
from sentence_transformers import SentenceTransformer

c:\Ambarish\AZURE-AI-VECTOR-SEARCH\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient, SearchIndexingBufferedSender  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryCaptionResult,
    QueryAnswerResult,
    SemanticErrorMode,
    SemanticErrorReason,
    SemanticSearchResultsType,
    QueryType,
    VectorizedQuery,
    VectorQuery,
    VectorFilterMode,    
)
from azure.search.documents.indexes.models import (  
    ExhaustiveKnnAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    SemanticPrioritizedFields,
    SemanticField,  
    SearchField,  
    SemanticSearch,
    VectorSearch,  
    HnswAlgorithmConfiguration,
    HnswParameters,  
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
    VectorSearchAlgorithmKind,
    VectorSearchProfile,
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    VectorSearch,
    ExhaustiveKnnParameters,
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    SemanticField,  
    SearchField,  
    VectorSearch,  
    HnswParameters,  
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
)  
  
# Configure environment variables  
load_dotenv()  
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT") 
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME") 
key = os.getenv("AZURE_SEARCH_ADMIN_KEY") 
model = os.getenv("MODEL_NAME")
credential = AzureKeyCredential(key)

In [3]:
model = SentenceTransformer(os.getenv("MODEL_NAME"))

In [4]:
query = "Segmentation"  
query_vector = model.encode([query])[0]
print(query_vector)

[ 2.30357256e-02  1.26671903e-02  4.44920026e-02 -3.89090069e-02
 -9.84340813e-03 -2.77223047e-02  1.16374157e-01 -3.10633257e-02
  7.35345855e-03 -7.42051154e-02  2.16113627e-02 -2.92581283e-02
 -2.43248362e-02  3.65417264e-02 -4.89201359e-02 -3.33300494e-02
 -3.51172686e-02  7.97075853e-02 -1.05214164e-01 -3.67749780e-02
 -7.42147444e-03 -1.16393575e-02 -6.64043501e-02 -1.18389372e-02
  4.79650088e-02  8.82574022e-02  3.63940671e-02  6.97665289e-03
  5.70957921e-02 -5.15103266e-02  3.25404964e-02  5.39209843e-02
  1.31803960e-01  1.92847028e-02 -1.00792320e-02 -4.30897884e-02
  2.85608079e-02  4.23089862e-02  4.69760317e-03  1.62016042e-02
 -9.48535427e-02 -4.78087254e-02  5.15721412e-03 -1.60938818e-02
  7.83037096e-02  2.15604194e-02 -5.93573041e-03  2.50712726e-02
  4.32969555e-02  1.07063809e-02 -7.33178332e-02 -3.93875092e-02
 -7.59633258e-02  3.81111689e-02 -8.46338645e-03  3.16928253e-02
  3.59484851e-02 -5.85748367e-02  7.93159083e-02  1.30122043e-02
  5.90134189e-02 -3.66817

### Vector Search

In [5]:
 
search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))  
vector_query = VectorizedQuery(vector=query_vector, 
                               k_nearest_neighbors=3, 
                               fields="embedding")

results = search_client.search(  
    search_text=None,  
    vector_queries=[vector_query],
    select=["line", "filename"],
    top=5
)  
  
for result in results:  
    print(f"Score: {result['@search.score']}")  
    print(f"Captions: {result['@search.captions']}")  
    print(f"Highlights: {result['@search.highlights']}")  
    print(f"Content: {result['line']}\n")  
    print(f"Filename: {result['filename']}\n")  
    print("###############################")

Score: 0.60312515
Captions: None
Highlights: None
Content: Segmentation in the body is first observed in which of the following:
(a) Platyhelminthes  (b) Aschelminthes  (c) Annelida  (d) Arthropoda
Rationalised 2023-24
54 BIOLOGY
14.Match the following:
(a)Operculum (i)Ctenophora
(b)Parapodia (ii)Mollusca
(c)Scales (iii)Porifera
(d)Comb plates (iv)Reptilia
(e)Radula (v)Annelida
(f)Hairs (vi)Cyclostomata and Chondrichthyes
(g)Choanocytes (vii) Mammalia
(h)Gill slits (viii) Osteichthyes
15

Filename: CHAP04-BIOLOGY-CLASS11.pdf

###############################
Score: 0.58866936
Captions: None
Highlights: None
Content: 3 Diagrammatic sectional view of :
(a) Coelomate (b) Pseudocoelomate
(c) AcoelomateThose animals in which the developing embryo has a thir d germinal layer ,
mesoderm , in between the ectoderm and endoderm, are called
triploblastic  animals (platyhelminthes to chordates, Figure 4.2b).
4.1.5 Segmentation
In some animals, the body is externally and internally divided into
segm

### Hybrid Search

In [6]:
search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))  
vector_query = VectorizedQuery(vector=query_vector, 
                               k_nearest_neighbors=3, 
                               fields="embedding")

results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["line", "filename"],
    top=3
)  
  
for result in results:  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['line']}\n")  
    print(f"Filename: {result['filename']}\n")  
    print("###############################")

Score: 0.03253968432545662
Content: Segmentation in the body is first observed in which of the following:
(a) Platyhelminthes  (b) Aschelminthes  (c) Annelida  (d) Arthropoda
Rationalised 2023-24
54 BIOLOGY
14.Match the following:
(a)Operculum (i)Ctenophora
(b)Parapodia (ii)Mollusca
(c)Scales (iii)Porifera
(d)Comb plates (iv)Reptilia
(e)Radula (v)Annelida
(f)Hairs (vi)Cyclostomata and Chondrichthyes
(g)Choanocytes (vii) Mammalia
(h)Gill slits (viii) Osteichthyes
15

Filename: CHAP04-BIOLOGY-CLASS11.pdf

###############################
Score: 0.032522473484277725
Content: 3 Diagrammatic sectional view of :
(a) Coelomate (b) Pseudocoelomate
(c) AcoelomateThose animals in which the developing embryo has a thir d germinal layer ,
mesoderm , in between the ectoderm and endoderm, are called
triploblastic  animals (platyhelminthes to chordates, Figure 4.2b).
4.1.5 Segmentation
In some animals, the body is externally and internally divided into
segments with a serial repetition of at least som

### Exhaustive KNN exact nearest neighbor search

In [7]:
search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))  
vector_query = VectorizedQuery(vector=query_vector, k_nearest_neighbors=3, 
                               fields="embedding",exhaustive=True)

results = search_client.search(  
    search_text=None,  
    vector_queries=[vector_query],
    select=["line", "filename"],
    top=3
)  
  
for result in results:  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['line']}\n")  
    print(f"Filename: {result['filename']}\n")  
    print("###############################")

Score: 0.60312515
Content: Segmentation in the body is first observed in which of the following:
(a) Platyhelminthes  (b) Aschelminthes  (c) Annelida  (d) Arthropoda
Rationalised 2023-24
54 BIOLOGY
14.Match the following:
(a)Operculum (i)Ctenophora
(b)Parapodia (ii)Mollusca
(c)Scales (iii)Porifera
(d)Comb plates (iv)Reptilia
(e)Radula (v)Annelida
(f)Hairs (vi)Cyclostomata and Chondrichthyes
(g)Choanocytes (vii) Mammalia
(h)Gill slits (viii) Osteichthyes
15

Filename: CHAP04-BIOLOGY-CLASS11.pdf

###############################
Score: 0.58866936
Content: 3 Diagrammatic sectional view of :
(a) Coelomate (b) Pseudocoelomate
(c) AcoelomateThose animals in which the developing embryo has a thir d germinal layer ,
mesoderm , in between the ectoderm and endoderm, are called
triploblastic  animals (platyhelminthes to chordates, Figure 4.2b).
4.1.5 Segmentation
In some animals, the body is externally and internally divided into
segments with a serial repetition of at least some organs

Filename:

### Semantic Hybrid Search

In [8]:
search_client = SearchClient(service_endpoint, 
                             index_name, 
                             AzureKeyCredential(key))  
vector_query = VectorizedQuery(vector=query_vector, 
                               k_nearest_neighbors=3, 
                               fields="embedding")

results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["line","filename"],
    query_type=QueryType.SEMANTIC, 
    semantic_configuration_name='my-semantic-config', 
    query_caption=QueryCaptionType.EXTRACTIVE, 
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=3
)  

In [9]:
semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

In [10]:
semantic_answers

[]

In [11]:
for result in results:
    print(f"Reranker Score: {result['@search.reranker_score']}")
    print(f"Content: {result['line']}\n")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

Reranker Score: 2.391352653503418
Content: 3 Diagrammatic sectional view of :
(a) Coelomate (b) Pseudocoelomate
(c) AcoelomateThose animals in which the developing embryo has a thir d germinal layer ,
mesoderm , in between the ectoderm and endoderm, are called
triploblastic  animals (platyhelminthes to chordates, Figure 4.2b).
4.1.5 Segmentation
In some animals, the body is externally and internally divided into
segments with a serial repetition of at least some organs

Caption: chap04-biology-class11.pdf. 3 diagrammatic sectional view of : (a) coelomate (b) pseudocoelomate (c) acoelomatethose animals in which the developing embryo has a thir d germinal layer , mesoderm , in between the ectoderm and endoderm, are called triploblastic  animals (platyhelminthes to chordates, figure 4.2b). 4.1.5<em> segmentation</em> in some …

Reranker Score: 2.342907428741455
Content: Segmentation in the body is first observed in which of the following:
(a) Platyhelminthes  (b) Aschelminthes  (c) Anneli